In [1]:
import torch
from feature_extractor import load_feature_extractor

In [2]:
model = load_feature_extractor(weight_path='./weights/twostream-resnet50_all_plain.pt')

In [3]:
x = torch.randn((6, 256, 256)).unsqueeze(0)
y = model(x)
y.shape

torch.Size([131072])

---

In [2]:
from pathlib import Path

folders = list(Path("C:/xBD/test_bldgs/").glob('*'))

In [3]:
import pandas as pd
df = pd.read_csv(list(folders[0].glob('*.csv'))[0], index_col=0)
df.head()

,coords,class
guatemala-volcano_00000003_post_disaster_0.png,"(465.94617850181066, 109.62942748980328)",minor-damage
guatemala-volcano_00000003_post_disaster_1.png,"(240.50073856928046, 314.8236608385315)",destroyed
guatemala-volcano_00000003_post_disaster_2.png,"(591.9558061636591, 266.25772937220216)",minor-damage
guatemala-volcano_00000005_post_disaster_0.png,"(814.0503282467604, 958.26675138958)",un-classified
guatemala-volcano_00000009_post_disaster_0.png,"(319.7156849146396, 745.2364736911228)",un-classified


In [14]:
df.iloc[0,0].replace('(','').replace(')','').split(',')

['465.94617850181066', ' 109.62942748980328']

In [3]:
pre = list(folders[0].glob('*pre_disaster*'))

In [4]:
post = list(folders[0].glob('*post_disaster*'))

In [5]:
pre[4]

WindowsPath('C:/xBD/test_bldgs/guatemala-volcano/guatemala-volcano_00000009_pre_disaster_0.png')

In [6]:
post[4]

WindowsPath('C:/xBD/test_bldgs/guatemala-volcano/guatemala-volcano_00000009_post_disaster_0.png')

---

In [3]:
import os
from pathlib import Path
from collections import defaultdict

disaster_dict = defaultdict(list)
xbd_path = 'C:/xBD'
subsets = ('/train_bldgs/', '/hold_bldgs/', '/test_bldgs/', '/tier3_bldgs/')

disaster_folders = os.listdir(xbd_path + subsets[0]) + os.listdir(xbd_path + subsets[-1])

for subset in subsets:
    for disaster in disaster_folders:
        disaster_dict[disaster + '_pre'].extend(map(str, Path(xbd_path + subset + disaster).glob('*pre_disaster*')))
        disaster_dict[disaster + '_post'].extend(map(str, Path(xbd_path + subset + disaster).glob('*post_disaster*')))
        disaster_dict[disaster + '_labels'].extend(map(str, Path(xbd_path + subset + disaster).glob('*.csv')))

In [4]:
disaster_dict.keys()

dict_keys(['guatemala-volcano_pre', 'guatemala-volcano_post', 'guatemala-volcano_labels', 'hurricane-florence_pre', 'hurricane-florence_post', 'hurricane-florence_labels', 'hurricane-harvey_pre', 'hurricane-harvey_post', 'hurricane-harvey_labels', 'hurricane-matthew_pre', 'hurricane-matthew_post', 'hurricane-matthew_labels', 'hurricane-michael_pre', 'hurricane-michael_post', 'hurricane-michael_labels', 'mexico-earthquake_pre', 'mexico-earthquake_post', 'mexico-earthquake_labels', 'midwest-flooding_pre', 'midwest-flooding_post', 'midwest-flooding_labels', 'palu-tsunami_pre', 'palu-tsunami_post', 'palu-tsunami_labels', 'santa-rosa-wildfire_pre', 'santa-rosa-wildfire_post', 'santa-rosa-wildfire_labels', 'socal-fire_pre', 'socal-fire_post', 'socal-fire_labels', 'joplin-tornado_pre', 'joplin-tornado_post', 'joplin-tornado_labels', 'lower-puna-volcano_pre', 'lower-puna-volcano_post', 'lower-puna-volcano_labels', 'moore-tornado_pre', 'moore-tornado_post', 'moore-tornado_labels', 'nepal-floodi

In [6]:
disaster_dict['hurricane-florence_pre'][:5]

['C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_pre_disaster_0.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_pre_disaster_1.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_pre_disaster_10.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_pre_disaster_100.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_pre_disaster_101.png']

In [7]:
disaster_dict['hurricane-florence_post'][:5]

['C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_post_disaster_0.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_post_disaster_1.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_post_disaster_10.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_post_disaster_100.png',
 'C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_00000000_post_disaster_101.png']

---

In [1]:
import json

with open('disaster_dirs.json', 'r') as JSON:
    disasters_dict = json.load(JSON)

In [3]:
for disaster in disasters_dict:
    print(disaster)
    break

guatemala-volcano_pre


In [4]:
disasters_dict['guatemala-volcano_labels']

['C:\\xBD\\train_bldgs\\guatemala-volcano\\guatemala-volcano_train_labels.csv',
 'C:\\xBD\\hold_bldgs\\guatemala-volcano\\guatemala-volcano_hold_labels.csv',
 'C:\\xBD\\test_bldgs\\guatemala-volcano\\guatemala-volcano_test_labels.csv']

In [5]:
disasters_dict['hurricane-florence_labels']

['C:\\xBD\\train_bldgs\\hurricane-florence\\hurricane-florence_train_labels.csv',
 'C:\\xBD\\hold_bldgs\\hurricane-florence\\hurricane-florence_hold_labels.csv',
 'C:\\xBD\\test_bldgs\\hurricane-florence\\hurricane-florence_test_labels.csv']

---

In [2]:
import pandas as pd
disaster = 'guatemala-volcano'
annotation_train = pd.read_csv(disasters_dict[disaster + '_labels'][0], index_col=0)
annotation_train['split'] = ['train'] * annotation_train.shape[0]
annotation_hold = pd.read_csv(disasters_dict[disaster + '_labels'][1], index_col=0)
annotation_hold['split'] = ['hold'] * annotation_hold.shape[0]
annotation_test = pd.read_csv(disasters_dict[disaster + '_labels'][2], index_col=0)
annotation_test['split'] = ['test'] * annotation_test.shape[0]
annotation = pd.concat((annotation_train, annotation_hold, annotation_test))

In [3]:
annotation.head()

,coords,class,split
guatemala-volcano_00000000_post_disaster_0.png,"(485.9121456599377, 215.77570850325483)",no-damage,train
guatemala-volcano_00000000_post_disaster_1.png,"(825.060485390974, 755.5210197985012)",no-damage,train
guatemala-volcano_00000000_post_disaster_2.png,"(1001.8022656179191, 840.8830078407719)",no-damage,train
guatemala-volcano_00000000_post_disaster_3.png,"(1015.6418526761435, 916.5294599748164)",no-damage,train
guatemala-volcano_00000000_post_disaster_4.png,"(995.5231309776162, 946.7686919466036)",no-damage,train


In [29]:
annotation.loc[['guatemala-volcano_00000000_post_disaster_0.png','guatemala-volcano_00000000_post_disaster_1.png'],:]

,coords,class
guatemala-volcano_00000000_post_disaster_0.png,"(485.9121456599377, 215.77570850325483)",no-damage
guatemala-volcano_00000000_post_disaster_1.png,"(825.060485390974, 755.5210197985012)",no-damage


In [6]:
list_post_images = disasters_dict[disaster + '_post']

In [14]:
import os
os.path.split(list_post_images[0])[1]

'guatemala-volcano_00000000_post_disaster_0.png'

In [15]:
annotation[os.path.split(list_post_images[0])[1]]

'no-damage'

In [16]:
annotation.value_counts()

no-damage        731
un-classified    178
destroyed         33
minor-damage      26
major-damage      23
Name: class, dtype: int64

In [20]:
label_en = {'no-damage': 0, 'minor-damage': 1, 'major-damage': 2, 'destroyed': 3}

In [30]:
l = list(annotation)
s = pd.Series(l).replace(label_en).values

---

In [9]:
import torch

def get_edge_weight(node1: torch.Tensor, node2: torch.Tensor):
    D = node1.shape[0]
    s = (torch.abs(node1 - node2)) / (torch.abs(node1) + torch.abs(node2))
    edge_weight = 1 - torch.sum(s)/D
    return edge_weight

get_edge_weight(torch.randn(5), torch.randn(5)).item()

0.10956364870071411

---

In [ ]:
from dataset import IIDxBD

xbd_path = "C:/xBD"
#xbd_path = 'D:/Datasets/xBD'
root = 'iidxbd_root'
data = IIDxBD(xbd_path=xbd_path, root=root)